In [1]:
import pandas as pd
import xarray as xr


In [2]:
from pangeo_forge_recipes.patterns import ConcatDim, FilePattern, pattern_from_file_sequence
from pangeo_forge_recipes.recipes import XarrayZarrRecipe,setup_logging


In [3]:
dates = pd.date_range('2009-07-01', '2009-07-10', freq='D')

In [4]:
dates

DatetimeIndex(['2009-07-01', '2009-07-02', '2009-07-03', '2009-07-04',
               '2009-07-05', '2009-07-06', '2009-07-07', '2009-07-08',
               '2009-07-09', '2009-07-10'],
              dtype='datetime64[ns]', freq='D')

In [5]:
url_base = (
    "https://ige-meom-opendap.univ-grenoble-alpes.fr"
    "/thredds/fileServer/meomopendap/extract/eNATL60/eNATL60-BLBT02/1d"
)

In [6]:
time_concat_dim = ConcatDim("time", dates, nitems_per_file=1)
time_concat_dim

ConcatDim(name='time', nitems_per_file=1)

In [7]:
def make_recipe(var,dep):
        input_url_pattern = url_base + "/eNATL60-BLBT02_y{time:%Y}m{time:%m}d{time:%d}.1d_"+var+dep+".nc"
        def make_url(time):
            return input_url_pattern.format(time=time)
        pattern = FilePattern(make_url, time_concat_dim)
        recipe = XarrayZarrRecipe(pattern, inputs_per_chunk=1)
        return recipe

In [8]:
recipe_mld=make_recipe('somxl010','')
recipe_mld

XarrayZarrRecipe(file_pattern=<FilePattern {'time': 10}>, storage_config=StorageConfig(target=FSSpecTarget(fs=<fsspec.implementations.local.LocalFileSystem object at 0x7f59d0170700>, root_path='/tmp/tmpxpjhwy39/ar45KGRE'), cache=CacheFSSpecTarget(fs=<fsspec.implementations.local.LocalFileSystem object at 0x7f59d0170700>, root_path='/tmp/tmpxpjhwy39/q4wE9hZq'), metadata=MetadataTarget(fs=<fsspec.implementations.local.LocalFileSystem object at 0x7f59d0170700>, root_path='/tmp/tmpxpjhwy39/9TZlFQNf')), inputs_per_chunk=1, target_chunks={}, cache_inputs=True, copy_input_to_local_file=False, consolidate_zarr=True, consolidate_dimension_coordinates=True, xarray_open_kwargs={}, xarray_concat_kwargs={}, delete_input_encoding=True, process_input=None, process_chunk=None, lock_timeout=None, subset_inputs={}, open_input_with_kerchunk=False)

In [9]:
setup_logging()

In [10]:
recipe_mld.file_pattern

<FilePattern {'time': 10}>

In [11]:
run_function = recipe_mld.to_function()

In [12]:
run_function()

pangeo_forge_recipes.recipes.xarray_zarr - INFO - Caching input 'Index({DimIndex(name='time', index=0, sequence_len=10, operation=<CombineOp.CONCAT: 2>)})'
pangeo_forge_recipes.storage - INFO - Caching file 'https://ige-meom-opendap.univ-grenoble-alpes.fr/thredds/fileServer/meomopendap/extract/eNATL60/eNATL60-BLBT02/1d/eNATL60-BLBT02_y2009m07d01.1d_somxl010.nc'
pangeo_forge_recipes.storage - INFO - Copying remote file 'https://ige-meom-opendap.univ-grenoble-alpes.fr/thredds/fileServer/meomopendap/extract/eNATL60/eNATL60-BLBT02/1d/eNATL60-BLBT02_y2009m07d01.1d_somxl010.nc' to cache
pangeo_forge_recipes.recipes.xarray_zarr - INFO - Caching input 'Index({DimIndex(name='time', index=1, sequence_len=10, operation=<CombineOp.CONCAT: 2>)})'
pangeo_forge_recipes.storage - INFO - Caching file 'https://ige-meom-opendap.univ-grenoble-alpes.fr/thredds/fileServer/meomopendap/extract/eNATL60/eNATL60-BLBT02/1d/eNATL60-BLBT02_y2009m07d02.1d_somxl010.nc'
pangeo_forge_recipes.storage - INFO - Copying re

/srv/conda/envs/notebook/lib/python3.9/site-packages/pangeo_forge_recipes/recipes/xarray_zarr.py:115: RuntimeWarning: Failed to open Zarr store with consolidated metadata, falling back to try reading non-consolidated metadata. This is typically much slower for opening a dataset. To silence this warning, consider:
1. Consolidating metadata in this existing store with zarr.consolidate_metadata().
2. Explicitly setting consolidated=False, to avoid trying to read consolidate metadata, or
3. Explicitly setting consolidated=True, to raise an error in this case instead of falling back to try reading non-consolidated metadata.
  return xr.open_zarr(target.get_mapper())


pangeo_forge_recipes.recipes.xarray_zarr - INFO - Storing dataset in /tmp/tmpxpjhwy39/ar45KGRE
pangeo_forge_recipes.recipes.xarray_zarr - INFO - Expanding target concat dim 'time' to size 10
pangeo_forge_recipes.recipes.xarray_zarr - INFO - Opening inputs for chunk Index({DimIndex(name='time', index=0, sequence_len=10, operation=<CombineOp.CONCAT: 2>)})
pangeo_forge_recipes.recipes.xarray_zarr - INFO - Opening input with Xarray Index({DimIndex(name='time', index=0, sequence_len=10, operation=<CombineOp.CONCAT: 2>)}): 'https://ige-meom-opendap.univ-grenoble-alpes.fr/thredds/fileServer/meomopendap/extract/eNATL60/eNATL60-BLBT02/1d/eNATL60-BLBT02_y2009m07d01.1d_somxl010.nc'
pangeo_forge_recipes.storage - INFO - Opening 'https://ige-meom-opendap.univ-grenoble-alpes.fr/thredds/fileServer/meomopendap/extract/eNATL60/eNATL60-BLBT02/1d/eNATL60-BLBT02_y2009m07d01.1d_somxl010.nc' from cache
pangeo_forge_recipes.recipes.xarray_zarr - INFO - Combining inputs for chunk 'Index({DimIndex(name='time',

In [13]:
mld_zarr_prune = xr.open_zarr(recipe_mld.target_mapper, consolidated=True)
mld_zarr_prune

<xarray.Dataset>
Dimensions:       (y: 4729, x: 8354, time_counter: 1)
Coordinates:
  * time_counter  (time_counter) datetime64[ns] 2009-07-01T12:00:00
Dimensions without coordinates: y, x
Data variables:
    nav_lat       (y, x) float32 dask.array<chunksize=(4729, 8354), meta=np.ndarray>
    nav_lon       (y, x) float32 dask.array<chunksize=(4729, 8354), meta=np.ndarray>
    somxl010      (time_counter, y, x) float32 dask.array<chunksize=(1, 4729, 8354), meta=np.ndarray>
Attributes:
    CASE:              N/A
    CONFIG:            N/A
    output_frequency:  N/A
    start_date:        -1